In [28]:
import os
import geopandas as gpd
import numpy as np
from osgeo import gdal
import rasterio
import rasterio.mask
#import netCDF4 as nc
import sys 


#Functions Imports 
sys.path.append(os.path.abspath("c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Functions"))
import Preprocess
#import ncdf_tiff

In [29]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\src\\data_preprocessing\\base_geojson')
mandal_boundaries = gpd.read_file('TS_mandal_boundary.json')
mandal_boundaries = mandal_boundaries.reset_index()

os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\src\\data_preprocessing\\base_geojson')
shape_telangana = gpd.read_file('TL_state_shapefile_for_clip.geojson')
shape_telangana = shape_telangana.explode()

In [30]:
import importlib
importlib.reload(Preprocess)
#importlib.reload(ncdf_tiff)


<module 'Preprocess' from 'c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Functions\\Preprocess.py'>

# Rewriting NETCDF Files
##### Add all NETCDF files that need to be rewritten into geotiff in Input Map, the geotiff will be saved in the Output Map.

In [31]:
#Choose Parameter

#PM2.5
parameter = ':GWRPM25'


In [22]:
ncdf_tiff.netcdf_to_tiff(parameter)

## Cropping Telangana Shape
#### Add all files that need to be cropped into the Output Map, the files in the Output Map will be replaced by the cropped files.

In [32]:
Preprocess.crop_image(shape_telangana)

RasterioIOError: c:/Users/Jesse/OneDrive/Documenten/Master BAOR/Thesis/GitHub/dicra/analytics/sandbox/notebooks/crop_fires/Preprocessing_Data_Dicra/Output/c_gls_BA300_202110100000_GLOBE_S3_V1.2.1/20211010: Permission denied

# Rescale the size if necessary

NO2

In [29]:
#First set all 255 values from NO2 to the following new data values
ndv = -9999

path = 'c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Output\\'
file_names = os.listdir(path) # contains all filenames
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Changed_NDV\\') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        Preprocess.del_rescale(i, ndv, tiff_path)
    except:
        print(i)

#For NO2, we need to rescale from 0-254 to 0-1500
scaling = 1500/254
offset = 0
ndv = -9999

path = 'c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Changed_NDV\\'
file_names = os.listdir(path) # contains all filenames
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Rescaled\\') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        Preprocess.rescale(i, ndv, scaling, offset, tiff_path)
    except:
        print(i)

LST

In [34]:
#First set all 255 values from Soil Temperature to the following new data values
ndv = -9999

path = 'c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Output\\'
file_names = os.listdir(path) # contains all filenames
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Changed_NDV\\') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        Preprocess.del_rescale(i, ndv, tiff_path)
    except:
        print(i)

#For Soil Temperature, we need to rescale from 0-254 to -25 - 45
scaling = 70/254
offset = -25
ndv = -9999

path = 'c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Changed_NDV\\'
file_names = os.listdir(path) # contains all filenames
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Rescaled\\') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        Preprocess.rescale(i, ndv, scaling, offset, tiff_path)
    except:
        print(i)

PM2.5

In [14]:
#For PM2.5 we do not need to rescale but only change the no data value to -9999
scaling = 1
offset = 0
ndv = -9999

path = 'c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Output\\'
file_names = os.listdir(path) # contains all filenames
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Rescaled\\') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        Preprocess.rescale(i, ndv, scaling, offset, tiff_path)
    except:
        print(i)

# COG
##### We need to create a geooptimized tif file. Add the following text in OSGeo4W Shell : 

#for %N in (C:/Users/Jesse/OneDrive/Documenten/Rescaled/*.tiff) DO gdal_translate C:/Users/Jesse/OneDrive/Documenten/Rescaled/%~nN.tiff  C:/Users/Jesse/OneDrive/Documenten/COG/%~nN.tiff -co COMPRESS=LZW -co TILED=YES


#OR
for %N in (C:/Users/Jesse/OneDrive/Documenten/Rescaled/*.tif) DO gdal_translate C:/Users/Jesse/OneDrive/Documenten/Rescaled/%~nN.tif  C:/Users/Jesse/OneDrive/Documenten/COG/%~nN.tif -co COMPRESS=LZW -co TILED=YES
 
